## Importar librerías

In [2]:
from data_functions import *
from model_factory import ConvBlockConfig, create_model_with_configurations
from custom_callbacks import get_callbacks
import json
import numpy as np
import matplotlib
import seaborn as sns
from sklearn.metrics import confusion_matrix

## Función de carga de datos.
Devuelve las particiones del dataset de train, test y validación

In [3]:
def load_data(gray_scale=False, augmenter=False, input_size=(224,224), batch_size=32):
    """Carga los tres datasets: train, test y valid

    Args:
        gray_scale (bool, optional): Dataset en escala de gris. Defaults to False.
        augmenter (bool, optional): Aumentar dataset en entrenamiento. Defaults to False.
        input_size (tuple, optional): Tamaño de entrada. Defaults to (224,224).
        batch_size (int, optional): Tamaño del batch. Defaults to 64.

    Returns:
        _type_: dataset
    """
    # Creates a dictionary mapping image paths to tuples containing class and partition.
    dict_dataset = get_dict_dataset()

    # Create dataframe
    df_dataset = dict2dataframe(dict_dataset)

    # Train
    df_dataset_train = df_dataset[df_dataset['split'] == 'Train'].drop(columns=['split'])

    # Valid
    df_dataset_valid = df_dataset[df_dataset['split'] == 'Valid'].drop(columns=['split'])

    # Test
    df_dataset_test = df_dataset[df_dataset['split'] == 'Test'].drop(columns=['split'])

    # Train dataset creation
    train_tfdataset = get_dataset(
        df=df_dataset_train,
        input_size=input_size,
        batch_size=batch_size,
        shuffle=True,
        gray_scale=gray_scale,
        augmenter=augmenter,
        num_aug=1,
    )

    # Valid dataset creation
    valid_tfdataset = get_dataset(
        df=df_dataset_valid,
        input_size=input_size,
        batch_size=batch_size,
        shuffle=True,
        gray_scale=gray_scale,
        augmenter=augmenter,
    )

    # Test dataset creation
    test_tfdataset = get_dataset(
        df=df_dataset_test,
        input_size=input_size,
        batch_size=batch_size,
        gray_scale=gray_scale,
        augmenter=augmenter,
        test_set=True,
    )


    # Calculamos los pesos de cada clase debido al desbalanceo de las clases
    computed_class_weights = compute_class_weight(class_weight='balanced', 
                                                  classes=np.unique(df_dataset_train['label'].values),
                                                  y=df_dataset_train['label'].values)
    class_weights = {idx: item for idx, item in enumerate(computed_class_weights)}

    # Num clases
    num_classes = df_dataset_train["label"].max() + 1

    return train_tfdataset, valid_tfdataset, test_tfdataset, class_weights, num_classes


## Bucle de entrenamiento
En este bucle se carga la factoría de modelos from scratch para ir probando cada uno y guardando sus resultados con los callbacks personalizados. Las configuraciones de los modelos vienen desde el archivo "scratch_experiments.json". Estas configuraciones se pasan a la factoría y se crean los modelos.

In [4]:
def scratch_train(input_shape, experiments_json:str="scratch_experiments.json"):
    # Cargar datos
    train_tfdataset, valid_tfdataset, test_tfdataset, class_weights, num_classes = load_data()

    # Cargar configuraciones desde el archivo JSON
    with open(experiments_json, 'r') as file:
        experiment_config = json.load(file)

    # Crear modelo, compilarlo y entrenarlo
    #history_list = []
    for experiment_name, config in experiment_config.items():
        print(f"Running Experiment: {experiment_name}")

        # Configurar el modelo utilizando las configuraciones
        block_configs = []
        for block_conf in config["model"]["conv_block_configs"]:
            block = ConvBlockConfig(block_conf["filters"], 
                                    block_conf["kernel_size"], 
                                    block_conf["use_maxpooling"],
                                    block_conf["use_batchnorm"],
                                    block_conf["dropout_value"],
                                    block_conf["name"])
            block_configs.append(block)
        
        # Crear modelo
        model = create_model_with_configurations(block_configs, input_shape, num_classes)
        
        # Compilar modelo
        model.compile(
            loss = tf.keras.losses.categorical_crossentropy,
            optimizer = tf.keras.optimizers.Adam(0.0001),
            metrics = ['acc']) 
        
        # Crear callbacks
        callbacks = get_callbacks(experiment_name)

        # Mostrar arquitectura
        print(model.summary())

        # Entrenar modelo
        history = model.fit(train_tfdataset.repeat(),
                            validation_data=valid_tfdataset,
                            class_weight = class_weights,
                            epochs = config["training"]["epochs"],
                            steps_per_epoch=config["training"]["steps_per_epoch"],
                            callbacks = callbacks,
                            verbose=1)
        

        #history_list.append(history)



## Lanzar entrenamiento
Usando la configuración de los modelos de la factoría, se lanzan todos los experimentos. De esta manera se ejecutan uno detrás de otro sin necesidad de estar pendiente de lanzarlos manualmente.

In [5]:
# Dimensiones de entrada
input_shape = (224, 224, 3)

# Lanzar entrenamiento
scratch_train(input_shape)

Number of instances per label: 
label
0     523
1     488
2     412
3     396
4     380
5     312
6     308
7     229
8     210
9     184
10    176
11    176
Name: count, dtype: int64

Percentaje of instances per label: 
label
0     0.137849
1     0.128624
2     0.108593
3     0.104375
4     0.100158
5     0.082235
6     0.081181
7     0.060358
8     0.055351
9     0.048498
10    0.046389
11    0.046389
Name: count, dtype: float64


2023-11-22 09:47:52.611296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 09:47:52.649197: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 09:47:52.649458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 09:47:52.652728: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 09:47:52.653060: I tensorflow/compile

 > Shuffle
 > Establecemos el batchsize
Number of instances per label: 
label
0     131
1     123
2     104
3     100
4      95
5      78
6      77
7      58
8      53
9      47
11     45
10     45
Name: count, dtype: int64

Percentaje of instances per label: 
label
0     0.137029
1     0.128661
2     0.108787
3     0.104603
4     0.099372
5     0.081590
6     0.080544
7     0.060669
8     0.055439
9     0.049163
11    0.047071
10    0.047071
Name: count, dtype: float64
 > Shuffle
 > Establecemos el batchsize
Number of instances per label: 
label
Unkown    794
Name: count, dtype: int64

Percentaje of instances per label: 
label
Unkown    1.0
Name: count, dtype: float64
 > Establecemos el batchsize
Running Experiment: model1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1 (Sequential)         (None, 112, 112, 64)      2048      
                                                

2023-11-22 09:47:54.504920: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/block1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-11-22 09:47:54.987519: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-22 09:47:55.464835: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-22 09:47:56.107108: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-22 09:47:57.037578: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x9b13c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-22 09:47:57.037620: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute

32/32 [==============================] - ETA: 0s - loss: 4.8663 - acc: 0.1553
Epoch 1: val_loss improved from inf to 14.95988, saving model to models/model1
INFO:tensorflow:Assets written to: models/model1/assets


...

32/32 [==============================] - 10s 326ms/step - loss: 0.4046 - acc: 0.8584 - val_loss: 0.7865 - val_acc: 0.7490
Epoch 39/50
32/32 [==============================] - ETA: 0s - loss: 0.3681 - acc: 0.8760
Epoch 39: val_loss did not improve from 0.78654
32/32 [==============================] - 9s 299ms/step - loss: 0.3681 - acc: 0.8760 - val_loss: 0.8759 - val_acc: 0.7040
Epoch 40/50
32/32 [==============================] - ETA: 0s - loss: 0.4090 - acc: 0.8613
Epoch 40: val_loss did not improve from 0.78654
32/32 [==============================] - 8s 269ms/step - loss: 0.4090 - acc: 0.8613 - val_loss: 0.9243 - val_acc: 0.6851
Epoch 41/50
32/32 [==============================] - ETA: 0s - loss: 0.3667 - acc: 0.8644
Epoch 41: val_loss improved from 0.78654 to 0.78521, saving model to models/model3_nobatchnorm
INFO:tensorflow:Assets written to: models/model3_nobatchnorm/assets


INFO:tensorflow:Assets written to: models/model3_nobatchnorm/assets


32/32 [==============================] - 9s 287ms/step - loss: 0.3667 - acc: 0.8644 - val_loss: 0.7852 - val_acc: 0.7500
Epoch 42/50
32/32 [==============================] - ETA: 0s - loss: 0.3558 - acc: 0.8779
Epoch 42: val_loss improved from 0.78521 to 0.75285, saving model to models/model3_nobatchnorm
INFO:tensorflow:Assets written to: models/model3_nobatchnorm/assets


INFO:tensorflow:Assets written to: models/model3_nobatchnorm/assets


32/32 [==============================] - 10s 326ms/step - loss: 0.3558 - acc: 0.8779 - val_loss: 0.7529 - val_acc: 0.7615
Epoch 43/50
32/32 [==============================] - ETA: 0s - loss: 0.3098 - acc: 0.9043
Epoch 43: val_loss did not improve from 0.75285
32/32 [==============================] - 9s 297ms/step - loss: 0.3098 - acc: 0.9043 - val_loss: 0.8475 - val_acc: 0.7364
Epoch 44/50
32/32 [==============================] - ETA: 0s - loss: 0.3633 - acc: 0.8711
Epoch 44: val_loss did not improve from 0.75285
32/32 [==============================] - 9s 279ms/step - loss: 0.3633 - acc: 0.8711 - val_loss: 0.9056 - val_acc: 0.7155
Epoch 45/50
32/32 [==============================] - ETA: 0s - loss: 0.3080 - acc: 0.8960
Epoch 45: val_loss did not improve from 0.75285
32/32 [==============================] - 9s 270ms/step - loss: 0.3080 - acc: 0.8960 - val_loss: 0.8206 - val_acc: 0.7374
Epoch 46/50
32/32 [==============================] - ETA: 0s - loss: 0.2831 - acc: 0.9053
Epoch 46: v